In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['CM']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('conditioned medium', 2364),
 ('conditioned media', 1363),
 ('cerebral malaria', 301),
 ('cardiomyocyte', 233),
 ('contrast media', 149),
 ('contingency management', 102),
 ('cutaneous melanoma', 75),
 ('chylomicron', 69),
 ('contrast medium', 60),
 ('cryptococcal meningitis', 56),
 ('childhood maltreatment', 55),
 ('cow s milk', 52),
 ('central memory', 51),
 ('chronic migraine', 47),
 ('cochlear microphonic', 44),
 ('cardiomyopathy', 43),
 ('chinese medicine', 42),
 ('cultured medium', 40),
 ('circular muscle', 37),
 ('case management', 36),
 ('cardiac myocytes', 29),
 ('cross metathesis', 29),
 ('cytoplasmic membrane', 29),
 ('cells membrane', 27),
 ('clinical mastitis', 23),
 ('chloramphenicol', 23),
 ('cultured media', 22),
 ('cordyceps militaris', 21),
 ('curcumin', 21),
 ('cytokine mixture', 19),
 ('calmodulin', 18),
 ('cap mesenchymal', 17),
 ('child maltreatment', 17),
 ('carboxymethyl', 17),
 ('chondrogenic medium', 17),
 ('chorismate mutase', 16),
 ('canola meal', 16),
 ('

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'MESH:D017077': 'Culture Media, Conditioned',
 'MESH:D016779': 'Malaria, Cerebral',
 'MESH:D032383': 'Myocytes, Cardiac',
 'CHEBI:CHEBI:37338': 'radioopaque medium',
 'DOID:DOID:8923': 'skin melanoma',
 'GO:GO:0042627': 'chylomicron',
 'MESH:D008892': 'Milk',
 'MESH:D008568': 'Memory',
 'MESH:D008881': 'Migraine Disorders',
 'MESH:D009202': 'Cardiomyopathies',
 'MESH:D008511': 'Medicine',
 'MESH:D009132': 'Muscles',
 'MESH:D019090': 'Case Management',
 'GO:GO:0005886': 'plasma membrane',
 'MESH:D008413': 'Mastitis',
 'CHEBI:CHEBI:17698': 'chloramphenicol',
 'CHEBI:CHEBI:3962': 'curcumin',
 'CHEBI:CHEBI:60004': 'mixture',
 'FPLX:CALM': 'CALM',
 'MESH:D002649': 'Child Abuse',
 'CHEBI:CHEBI:41402': 'carboxymethyl group',
 'MESH:D062407': 'Meals',
 'EFO:0000579': 'media',
 'MESH:D024722': 'Cardiac Myosins',
 'CHEBI:CHEBI:48544': 'methanesulfonates',
 'HP:HP:0011672': 'Cardiac myxoma',
 'MESH:C071162': 'citrulline malate',
 'CHEBI:CHEBI:3699': 'cimetidine',
 'DOID:DOID:1751': 'malignant co

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-10 01:25:50] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'c militaris': 'TAXONOMY:73501',
  'caga multimerization': 'ungrounded',
  'calcification medium': 'ungrounded',
  'calmodulin': 'FPLX:CALM',
  'camel milk': 'ungrounded',
  'camelina meal': 'ungrounded',
  'camostat mesilate': 'CHEBI:CHEBI:135632',
  'camostat methyl': 'CHEBI:CHEBI:135632',
  'canola meal': 'ungrounded',
  'cap mesenchymal': 'ungrounded',
  'carbohydrate metabolism': 'MESH:D050260',
  'carbon material': 'ungrounded',
  'carboxymethyl': 'ungrounded',
  'cardiac memory': 'ungrounded',
  'cardiac mesoderm': 'EFO:0000315',
  'cardiac metastases': 'ungrounded',
  'cardiac muscle': 'MESH:D009206',
  'cardiac myocytes': 'NCIT:C13002',
  'cardiac myosin': 'MESH:D024722',
  'cardiac myxoma': 'HP:HP:0011672',
  'cardiometabolic': 'ungrounded',
  'cardiomyocyte': 'NCIT:C13002',
  'cardiomyopathic': 'ungrounded',
  'cardiomyopathy': 'NCIT:C34830',
  'carrier multiple': 'ungrounded',
  'case management': 'MESH:D019090',
  'cassia mimosoides var nomame makino': 'ungrounded',
  'c

In [12]:
grounding_map, names, pos_labels = [{'c militaris': 'TAXONOMY:73501',
  'caga multimerization': 'ungrounded',
  'calcification medium': 'ungrounded',
  'calmodulin': 'FPLX:CALM',
  'camel milk': 'ungrounded',
  'camelina meal': 'ungrounded',
  'camostat mesilate': 'CHEBI:CHEBI:135632',
  'camostat methyl': 'CHEBI:CHEBI:135632',
  'canola meal': 'ungrounded',
  'cap mesenchymal': 'ungrounded',
  'carbohydrate metabolism': 'MESH:D050260',
  'carbon material': 'ungrounded',
  'carboxymethyl': 'ungrounded',
  'cardiac memory': 'ungrounded',
  'cardiac mesoderm': 'EFO:0000315',
  'cardiac metastases': 'ungrounded',
  'cardiac muscle': 'MESH:D009206',
  'cardiac myocytes': 'NCIT:C13002',
  'cardiac myosin': 'MESH:D024722',
  'cardiac myxoma': 'HP:HP:0011672',
  'cardiometabolic': 'ungrounded',
  'cardiomyocyte': 'NCIT:C13002',
  'cardiomyopathic': 'NCIT:C34830',
  'cardiomyopathy': 'NCIT:C34830',
  'carrier multiple': 'ungrounded',
  'case management': 'MESH:D019090',
  'cassia mimosoides var nomame makino': 'ungrounded',
  'cattle manure': 'ungrounded',
  'cavernous malformation': 'ungrounded',
  'cbfβ smmhc': 'ungrounded',
  'cells membrane': 'GO:GO:0005886',
  'cells membrane protein': 'MESH:D008565',
  'cellulose membrane': 'ungrounded',
  'central medial': 'ungrounded',
  'central medial thalamus nucleus': 'ungrounded',
  'central memory': 'ungrounded',
  'centromedial': 'ungrounded',
  'ceramide': 'CHEBI:CHEBI:17761',
  'cerebral malaria': 'NCIT:C128373',
  'cerebral malaria alone': 'NCIT:C128373',
  'cerebral malaria patients': 'NCIT:C128373',
  'cervical mucus': 'MESH:D002582',
  'chiari like malformation': 'MESH:D001139',
  'chiari malformation': 'MESH:D001139',
  'chicken manure': 'ungrounded',
  'child maltreatment': 'MESH:D002649',
  'childhood maltreatment': 'MESH:D002649',
  'chinensis plus myrobalan': 'ungrounded',
  'chinese medicine': 'ungrounded',
  'chitosan microparticles': 'ungrounded',
  'chloramphenicol': 'CHEBI:CHEBI:17698',
  'chloroform methanolic': 'ungrounded',
  'chloromethane': 'CHEBI:CHEBI:36014',
  'chocolate milk': 'ungrounded',
  'chondrogenic media': 'ungrounded',
  'chondrogenic medium': 'ungrounded',
  'chorismate mutase': 'MESH:D002826',
  'choroidal melanoma': 'HP:HP:0012054',
  'chronic migraine': 'NCIT:C117015',
  'chylomicron': 'GO:GO:0042627',
  'ciliary margin': 'ungrounded',
  'ciliary muscle': 'ungrounded',
  'cimetidine': 'CHEBI:CHEBI:3699',
  'circular muscle': 'ungrounded',
  'circular smooth muscle': 'ungrounded',
  'cisterna magna': 'MESH:D002946',
  'citrulline malate': 'MESH:C071162',
  'clarithromycin': 'CHEBI:CHEBI:3732',
  'clinical mastitis': 'DOID:DOID:10690',
  'cm': 'ungrounded',
  'cm treated': 'ungrounded',
  'cocaine methiodide': 'MESH:C076070',
  'coccidioidal meningitis': 'ungrounded',
  'cochlear microphonic': 'ungrounded',
  'cochlear microphonic potential': 'MESH:D003055',
  'collagen matrix': 'ungrounded',
  'collagen membrane': 'ungrounded',
  'combretum micranthum extract': 'ungrounded',
  'common marmoset': 'MESH:D002144',
  'compassion meditation': 'ungrounded',
  'complementary medicine': 'ungrounded',
  'complete media': 'MESH:D003470',
  'complete medium': 'MESH:D003470',
  'complete mole': 'ungrounded',
  'condi tion medium': 'ungrounded',
  'conditioned media': 'MESH:D017077',
  'conditioned medium': 'MESH:D017077',
  'confocal microscopy': 'MESH:D018613',
  'congenital malformation': 'ungrounded',
  'conjunctival melanoma': 'MESH:D034701',
  'conserved motif': 'ungrounded',
  'contaminant mixture': 'ungrounded',
  'contingency management': 'ungrounded',
  'continuous motion': 'ungrounded',
  'contrast material': 'ungrounded',
  'contrast media': 'CHEBI:CHEBI:37338',
  'contrast medium': 'CHEBI:CHEBI:37338',
  'control media': 'ungrounded',
  'control medium': 'ungrounded',
  'conventional management practices': 'ungrounded',
  'conventional medicine': 'ungrounded',
  'conventional method': 'ungrounded',
  'convoluted membrane': 'ungrounded',
  'cordyceps militaris': 'TAXONOMY:73501',
  'corticomotoneuronal': 'ungrounded',
  'corticosterone metabolite': 'ungrounded',
  'coumermycin': 'MESH:C004628',
  'cow manure': 'ungrounded',
  'cow milk': 'MESH:D008892',
  'cow s milk': 'MESH:D008892',
  'cow s milk protein': 'MESH:D008894',
  'creatine monohydrate': 'ungrounded',
  'cribriform morphology': 'ungrounded',
  'cristae membrane': 'ungrounded',
  'cross metathesis': 'ungrounded',
  'crude membrane': 'ungrounded',
  'crude methanolic extract': 'ungrounded',
  'cryptococcal meningitis': 'HP:HP:0032160',
  'cryptococcal meningoencephalitis': 'MESH:D008590',
  'cultured media': 'MESH:D003470',
  'cultured medium': 'MESH:D003470',
  'curcumin': 'ungrounded',
  'cutaneous mastocytosis': 'MESH:D034701',
  'cutaneous maximus': 'ungrounded',
  'cutaneous melanoma': 'NCIT:C3510',
  'cyclic meditation': 'ungrounded',
  'cymbopogon martinii': 'ungrounded',
  'cynomolgus': 'MESH:D008252',
  'cynomolgus monkey': 'MESH:D008252',
  'cypermethrin': 'CHEBI:CHEBI:4042',
  'cytokine mix': 'ungrounded',
  'cytokine mixture': 'ungrounded',
  'cytomodulin': 'ungrounded',
  'cytoplasmic membrane': 'GO:GO:0005886',
  'm brassicae': 'ungrounded',
  'mamestra brassicae': 'ungrounded',
  'media conditioned': 'MESH:D017077',
  'media were collected': 'ungrounded',
  'medium conditioned': 'MESH:D017077',
  'medium conditioned with oa cartilage': 'MESH:D017077'},
 {'TAXONOMY:73501': 'Cordyceps militaris',
  'FPLX:CALM': 'CALM',
  'CHEBI:CHEBI:135632': 'camostat',
  'MESH:D050260': 'Carbohydrate Metabolism',
  'EFO:0000315': 'cardiac mesoderm primordium',
  'MESH:D009206': 'Myocardium',
  'NCIT:C13002': 'Cardiomyocyte',
  'MESH:D024722': 'Cardiac Myosins',
  'HP:HP:0011672': 'Cardiac myxoma',
  'NCIT:C34830': 'Cardiomyopathy',
  'MESH:D019090': 'Case Management',
  'GO:GO:0005886': 'plasma membrane',
  'MESH:D008565': 'Membrane Proteins',
  'CHEBI:CHEBI:17761': 'ceramide',
  'NCIT:C128373': 'Cerebral Malaria',
  'MESH:D002582': 'Cervix Mucus',
  'MESH:D001139': 'Arnold-Chiari Malformation',
  'MESH:D002649': 'Child Abuse',
  'CHEBI:CHEBI:17698': 'chloramphenicol',
  'CHEBI:CHEBI:36014': 'chloromethane',
  'MESH:D002826': 'Chorismate Mutase',
  'HP:HP:0012054': 'Choroidal melanoma',
  'NCIT:C117015': 'Chronic Migraine',
  'GO:GO:0042627': 'chylomicron',
  'CHEBI:CHEBI:3699': 'cimetidine',
  'MESH:D002946': 'Cisterna Magna',
  'MESH:C071162': 'citrulline malate',
  'CHEBI:CHEBI:3732': 'clarithromycin',
  'DOID:DOID:10690': 'mastitis',
  'MESH:C076070': 'cocaine methiodide',
  'MESH:D003055': 'Cochlear Microphonic Potentials',
  'MESH:D002144': 'Callithrix',
  'MESH:D003470': 'Culture Media',
  'MESH:D017077': 'Culture Media, Conditioned',
  'MESH:D018613': 'Microscopy, Confocal',
  'MESH:D034701': 'Mastocytosis, Cutaneous',
  'CHEBI:CHEBI:37338': 'radioopaque medium',
  'MESH:C004628': 'coumermycin',
  'MESH:D008892': 'Milk',
  'MESH:D008894': 'Milk Proteins',
  'HP:HP:0032160': 'Cryptococcal meningitis',
  'MESH:D008590': 'Meningoencephalitis',
  'NCIT:C3510': 'Cutaneous Melanoma',
  'MESH:D008252': 'Macaca fascicularis',
  'CHEBI:CHEBI:4042': 'cypermethrin'},
 ['CHEBI:CHEBI:135632',
  'FPLX:CALM',
  'GO:GO:0042627',
  'NCIT:C128373',
  'NCIT:C3510']]

In [13]:
excluded_longforms = ['cm', 'cm treated']

In [14]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = {}

In [15]:
unambiguous_agent_texts = {}

In [16]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

NameError: name 'additional_entities' is not defined

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [17]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-10 02:23:10] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-10 02:26:48] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9158635686479375 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [18]:
classifier.stats

{'label_distribution': {'CHEBI:CHEBI:17698': 15,
  'CHEBI:CHEBI:3699': 6,
  'MESH:D003470': 70,
  'MESH:D017077': 2295,
  'ungrounded': 472,
  'CHEBI:CHEBI:37338': 171,
  'NCIT:C128373': 176,
  'NCIT:C34830': 37,
  'MESH:D002582': 6,
  'FPLX:CALM': 16,
  'GO:GO:0042627': 59,
  'MESH:D003055': 5,
  'NCIT:C13002': 201,
  'GO:GO:0005886': 40,
  'MESH:D008892': 39,
  'MESH:D002826': 14,
  'DOID:DOID:10690': 21,
  'CHEBI:CHEBI:17761': 4,
  'NCIT:C3510': 53,
  'MESH:D002946': 4,
  'MESH:D008894': 2,
  'CHEBI:CHEBI:135632': 10,
  'CHEBI:CHEBI:4042': 4,
  'MESH:C071162': 6,
  'MESH:D024722': 7,
  'NCIT:C117015': 33,
  'MESH:C076070': 2,
  'EFO:0000315': 3,
  'CHEBI:CHEBI:36014': 2,
  'HP:HP:0012054': 6,
  'CHEBI:CHEBI:3732': 3,
  'MESH:D002144': 3,
  'MESH:D019090': 17,
  'HP:HP:0032160': 39,
  'MESH:D008252': 10,
  'MESH:C004628': 4,
  'TAXONOMY:73501': 14,
  'MESH:D034701': 9,
  'MESH:D002649': 46,
  'MESH:D001139': 9,
  'HP:HP:0011672': 8,
  'MESH:D008590': 3,
  'MESH:D050260': 2,
  'MESH:D

In [19]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [20]:
disamb.dump(model_name, results_path)

In [21]:
print(disamb.info())

Disambiguation model for CM

Produces the disambiguations:
	Arnold-Chiari Malformation	MESH:D001139
	CALM*	FPLX:CALM
	Callithrix	MESH:D002144
	Carbohydrate Metabolism	MESH:D050260
	Cardiac Myosins	MESH:D024722
	Cardiac myxoma	HP:HP:0011672
	Cardiomyocyte	NCIT:C13002
	Cardiomyopathy	NCIT:C34830
	Case Management	MESH:D019090
	Cerebral Malaria*	NCIT:C128373
	Cervix Mucus	MESH:D002582
	Child Abuse	MESH:D002649
	Chorismate Mutase	MESH:D002826
	Choroidal melanoma	HP:HP:0012054
	Chronic Migraine	NCIT:C117015
	Cisterna Magna	MESH:D002946
	Cochlear Microphonic Potentials	MESH:D003055
	Cordyceps militaris	TAXONOMY:73501
	Cryptococcal meningitis	HP:HP:0032160
	Culture Media	MESH:D003470
	Culture Media, Conditioned	MESH:D017077
	Cutaneous Melanoma*	NCIT:C3510
	Macaca fascicularis	MESH:D008252
	Mastocytosis, Cutaneous	MESH:D034701
	Membrane Proteins	MESH:D008565
	Meningoencephalitis	MESH:D008590
	Microscopy, Confocal	MESH:D018613
	Milk	MESH:D008892
	Milk Proteins	MESH:D008894
	Myocardium	MESH:D0092

In [22]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1